# New Section

In [0]:
!mkdir tensorboard_tut

In [0]:
cd tensorboard_tut

/content/tensorboard_tut


In [0]:
!mkdir summary_log

In [0]:
ls 

In [0]:
pwd

'/content/tensorboard_tut'

### Tensorboard Installation

In [0]:
LOG_DIR = '/content/tensorboard_tut/summary_log/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
"""Make sure we're able to connect to the TensorBoard service."""
! curl http://localhost:6006

In [0]:
"""## 2
Download and unzip [ngrok](https://ngrok.com).
"""
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1


In [0]:
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
! ls

ngrok  ngrok-stable-linux-amd64.zip


In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


https://6edadc2a.ngrok.io


In [0]:
# Install
! npm install -g localtunnel

# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

# Get url
!cat url.txt

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/client
+ localtunnel@1.9.1
added 55 packages from 34 contributors in 2.746s


### Sample Code to test tensorboard

In [0]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)

x = tf.add(3,5)

#using with clause 

with tf.Session() as sess:
  writer = tf.summary.FileWriter("/content/tensorboard_tut/summary_log/", sess.graph)
  print ("value of x : ",sess.run(x))
  
writer.close()

value of x :  8


### Link to Tensorboard

In [0]:
ls /content/tensorboard_tut/summary_log/

events.out.tfevents.1553955154.07e8a44c673d


In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://6edadc2a.ngrok.io


# Main tutorial

[link text](https://thecodacus.com/tensorboard-tutorial-visualize-networks-graphically/#.XJ9GzHUzYUg)

**Basic Concepts To Know To Use Tensor Board**

To use tensorboard we need to learn some basic concepts. these are as follows

* Summary & Summary Writer
* Scope


**Summary & Summary Writer**

we can easily see the histogram of the weights of a layer in tensorboard beautifully displayed.

So lets see how we can use  Summary & Summary Writer to do all these


In [0]:
inputs=tf.placeholder('float',[None,2],name='Input')
targets=tf.placeholder('float',name='Target')

In [0]:
weight1=tf.Variable(tf.random_normal(shape=[2,3],stddev=0.02),name="Weight1")
biases1=tf.Variable(tf.random_normal(shape=[3],stddev=0.02),name="Biases1")
tf.summary.histogram("weight/1",weight1)

Here what we did is we created a histogram summary using tf.summary.histogram() for us to read. so in tensor board we will be able to see the histogram data of the weights. The first argument is the name for that plot and second is the value for the plot (it’s better to use underscore “_” in the name to differentiate similar plots, we will see why that’s useful).



In [0]:
hLayer=tf.matmul(inputs,weight1)
hLayer=hLayer+biases1
hLayer=tf.sigmoid(hLayer, name='hActivation')

And for the next layer let’s do the same


In [0]:
weight2=tf.Variable(tf.random_normal(shape=[3,1],stddev=0.02),name="Weight2")
biases2=tf.Variable(tf.random_normal(shape=[1],stddev=0.02),name="Biases2")
tf.summary.histogram("weight/2",weight2)

In [0]:
output=tf.matmul(hLayer,weight2)
output=output+biases2
output=tf.sigmoid(output, name='outActivation')

## **Scalar Summary For Cost**

**We will create the error/cost calculation node and we added a summary for the cost using tf.summary.scalar(“cost”, cost) **

In [0]:
cost=tf.squared_difference(targets ,output)
cost=tf.reduce_mean(cost)
tf.summary.scalar("cost", cost)

optimizer=tf.train.AdamOptimizer().minimize(cost)

**So why scalar summary for cost and histogram summary for weights?**

Well scalar summary works with single values and it plots it in a 2D (x, y) coordinates with x as time steps and y as the value of the scalar. Where as we use histogram when there is a collection of data and we don’t want the plot for each and every value, or sometime it’s not possible to do so if the collection contains thousands of values. In those cases we want to see how the data is distributed. Histogram shows the distribution of the data

In [0]:
#generating inputs
import numpy as np

inp=[[0,0],[0,1],[1,0],[1,1]]
out=[[0],[1],[1],[0]]

inp=np.array(inp)
out=np.array(out)

## **Now lets start the session**

still now we only described tensorflow what to write in the summary. now inside the session we are actually going to write it

In [0]:
epochs=4000 # number of time we want to repeat
import datetime
with tf.Session() as sess:
    summaryMerged = tf.summary.merge_all()
    filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
    writer = tf.summary.FileWriter(filename, sess.graph)
    tf.global_variables_initializer().run()
    for i in range(epochs):
        error,_,sumOut =sess.run([cost,optimizer,summaryMerged],feed_dict={inputs: inp,targets:out})
        # print(i,error) # we dont need this anymore
        writer.add_summary(sumOut,i)

In the above code we started a session with name sess, we merged all the summary in a single summary node using tf.summary.merge_all() method, and named it summaryMerged.


Next we needed a summary writer which will write the summary in a file and our tensorboard will read that file and will update the summary on the screen. So we created a file path to store the summary file. I added a time stamp at the end of the file name. That way we don’t have to delete the old summary file every time we re run the script. (Otherwise all the data of the previous runs will be added in the same plots and it will become a mess )

So we created a writer using tf.summary.FileWriter(filename, sess.graph)  first argument is the filename, and next is the session graph. We are writing the graph of the session using sess.graph, which means the layout of the neural network. we will see that soon.


Now in the sess.run() inside the for loop we are evaluating the summaryMerged  node and the output we got in the sumOut  variable.


finally we are writing that in the file using the writer object writer.add_summary(sumOut,i) , the first argument is the summary output and the second argument is the time step in this case we used iteration number i




### Link to Tensorboard

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# RESET RUNTIME BEFORE YOU RUN FOLLOWING SECTION

# What is Scopes


So scope is basically grouping a set of operations and giving them a name, So that in the graph we can see that set of operation inside that particular scope block,

Ok lets modify our previous code with scope,



In [0]:
#generating inputs
import numpy as np

inp=[[0,0],[0,1],[1,0],[1,1]]
out=[[0],[1],[1],[0]]

inp=np.array(inp)
out=np.array(out)

In [0]:
with tf.name_scope("Input") as scope:
    inputs=tf.placeholder('float',[None,2],name='Input')

with tf.name_scope("Target") as scope:
    targets=tf.placeholder('float',name='Target')

So we put inputs placeholder inside a scope named “Input” and the targets placeholder inside a scope named “Target

Lets do the same thing with the rest of the code
with

In [0]:
with tf.name_scope("Hidden_Layer") as scope:
    weight1=tf.Variable(tf.random_normal(shape=[2,3],stddev=0.02),name="Weight1")
    biases1=tf.Variable(tf.random_normal(shape=[3],stddev=0.02),name="Biases1")
    tf.summary.histogram("weight_1",weight1)

    hLayer=tf.matmul(inputs,weight1)
    hLayer=hLayer+biases1
    hLayer=tf.sigmoid(hLayer, name='hActivation')

with tf.name_scope("Output_Layer") as scope:
    weight2=tf.Variable(tf.random_normal(shape=[3,1],stddev=0.02),name="Weight2")
    biases2=tf.Variable(tf.random_normal(shape=[1],stddev=0.02),name="Biases2")
    tf.summary.histogram("weight_2",weight2)

    output=tf.matmul(hLayer,weight2)
    output=output+biases2
    output=tf.sigmoid(output, name='outActivation')

with tf.name_scope("Optimization_Block") as scope:
    cost=tf.squared_difference(targets ,output)
    cost=tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

    optimizer=tf.train.AdamOptimizer().minimize(cost)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
epochs=4000 # number of time we want to repeat
import datetime

      
          
with tf.Session() as sess:
    summaryMerged = tf.summary.merge_all()
    filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
    writer = tf.summary.FileWriter(filename, sess.graph)
    tf.global_variables_initializer().run()
    for i in range(epochs):
        error,_,sumOut =sess.run([cost,optimizer,summaryMerged],feed_dict={inputs: inp,targets:out})
        # print(i,error) # we dont need this anymore
        writer.add_summary(sumOut,i)          

# TensorBoard in detail

## visualize the graph in TensorBoard

export the TensorFlow operations into a file, called event file (or event log file). TensorBoard is able to read this file and give insight into the model graph and its performance.


To visualize the program with TensorBoard, we need to write log files of the program. To write event files, we first need to create a writer for those logs, using this code:

```
writer = tf.summary.FileWriter([logdir], [graph])
 
```
where [logdir] is the folder where you want to store those log files. he second argument [graph] is the graph of the program we're working on. There are two ways to get the graph:

1. Call the graph using tf.get_default_graph(), which returns the default graph of the program

2. set it as sess.graph which returns the session's graph (note that this requires us to already have created a session).


We'll show both ways in the following example; however, the second way is more common. **Either way, make sure to create a writer only after you’ve defined your graph**. Otherwise, the graph visualized on TensorBoard would be incomplete.



In [0]:
import tensorflow as tf
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create graph
a = tf.constant(2)
b = tf.constant(3)
c = tf.add(a, b)

# creating the writer out of the session
# writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

# launch the graph in a session
with tf.Session() as sess:
    # or creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(c))
 

**Note **: If you run your code several times with the same [logdir], there will be multiple event files in your [logdir]. TF will show only the latest graph and display the warning of multiple event files. To get rid of the warning, delete the event files you no longer need or save them in different [logdir] folders

## Writing Summaries to Visualize Learning

So far we only focused on how to visualize the graph in TensorBoard. In this second part, we are now going to use a special operation called summary to visualize the model parameters (like weights and biases of a neural network), metrics (like loss or accuracy value), and images (like input images to a network).

Summary is a special TensorBoard operation that takes in a regular tenor and outputs the summarized data to your disk (i.e. in the event file). 

Basically, there are three main types of summaries:

**1. tf.summary.scalar: **used to write a single scalar-valued tensor (like classificaion loss or accuracy value)

**2. tf.summary.histogram: **used to plot histogram of all the values of a non-scalar tensor (like weight or bias matrices of a neural network)

**3. tf.summary.image:** used to plot images (like input images of a network, or generated output images of an autoencoder or a GAN)

In the following sections, we'll go through each of the above summary types in more details.



### 1. tf.summary.scalar:

It's for writing the values of a scalar tensor that changes over time or iterations. In the case of neural networks (say a simple network for classification task), it's usually used to monitor the changes of loss function or classification accuracy.



---

[link text](https://www.easy-tensorflow.com/tf-tutorials/basics/introduction-to-tensorboard)

In [0]:
import tensorflow as tf
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create the scalar variable
x_scalar = tf.get_variable('x_scalar', shape=[], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# ____step 1:____ create the scalar summary
first_summary = tf.summary.scalar(name='My_first_scalar_summary', tensor=x_scalar)

init = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # ____step 2:____ creating the writer inside the session
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    for step in range(100):
        # loop over several initializations of the variable
        sess.run(init)
        # ____step 3:____ evaluate the scalar summary
        summary = sess.run(first_summary)
        # ____step 4:____ add the summary to the writer (i.e. to the event file)
        writer.add_summary(summary, step)
    print('Done with writing the scalar summary')